In [2]:
#Insatlling necessary libraries
!pip install yfinance
!pip install plotly

     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ---- --------------------------------- 122.9/948.2 kB 2.4 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 2.8 MB/s eta 0:00:01
     --------------------- ---------------- 532.5/948.2 kB 3.7 MB/s eta 0:00:01
     --------------------------- ---------- 686.1/948.2 kB 3.6 MB/s eta 0:00:01
     ----------------------------------- -- 890.9/948.2 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 948.2/948.2 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/104.7 kB ? eta -:--:--
   ---------------------------------------- 10


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/19.1 MB 1.3 MB/s eta 0:00:15
   ---------------------------------------- 0.2/19.1 MB 2.1 MB/s eta 0:00:10
    --------------------------------------- 0.3/19.1 MB 1.8 MB/s eta 0:00:11
   - -------------------------------------- 0.7/19.1 MB 3.2 MB/s eta 0:00:06
   - -------------------------------------- 0.9/19.1 MB 3.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/19.1 MB 3.7 MB/s eta 0:00:05
   -- ------------------------------------- 1.4/19.1 MB 3.9 MB/s eta 0:00:05
   --- ------------------------------------ 1.5/19.1 MB 3.8 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/19.1 MB 3.7 MB/s eta 0:00:05
   --- ------------------------------------ 1.8/19.1 MB 3.7 MB/s eta 0:00:05
   ---- ----------------------------------- 2.0/19.1 MB 3.8 MB/s eta 0:00:05
   ---- ----

In [3]:
# Importing required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go


In [4]:
# Downloading stock data
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [5]:
# Calculating moving averages and generating buy/sell signals
def moving_average_crossover(data, short_window, long_window):
    signals = pd.DataFrame(index=data.index)
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

    # Create signals using .iloc for positional slicing
    signals['signal'] = 0.0
    signals.iloc[short_window:, signals.columns.get_loc('signal')] = np.where(
        signals['short_mavg'].iloc[short_window:] > signals['long_mavg'].iloc[short_window:], 1.0, 0.0
    )
    signals['positions'] = signals['signal'].diff()

    return signals

In [6]:
# Backtesting strategy and generating portfolio performance
def backtest_strategy(data, signals, initial_capital=100000):
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['positions'] = signals['positions']
    portfolio['market_price'] = data['Close']
    portfolio['cash'] = initial_capital
    portfolio['holding'] = 0.0
    portfolio['total'] = initial_capital

    position = 0.0
    cash = initial_capital

    for i in range(len(portfolio)):
        price = portfolio['market_price'].iloc[i]

        # Buy signal
        if signals['positions'].iloc[i] == 1.0:
            position = cash / price
            cash = 0.0

        # Sell signal
        elif signals['positions'].iloc[i] == -1.0:
            cash += position * price
            position = 0.0

        portfolio.iloc[i, portfolio.columns.get_loc('cash')] = cash
        portfolio.iloc[i, portfolio.columns.get_loc('holding')] = position * price
        portfolio.iloc[i, portfolio.columns.get_loc('total')] = cash + (position * price)

    # Plotting portfolio value
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=portfolio.index, y=portfolio['total'], mode='lines', name='Total Portfolio Value'))
    fig.add_trace(go.Scatter(x=portfolio[signals['positions'] == 1.0].index,
                             y=portfolio['total'][signals['positions'] == 1.0],
                             mode='markers', name='Buy', marker=dict(color='green', size=10)))
    fig.add_trace(go.Scatter(x=portfolio[signals['positions'] == -1.0].index,
                             y=portfolio['total'][signals['positions'] == -1.0],
                             mode='markers', name='Sell', marker=dict(color='red', size=10)))
    fig.update_layout(title=f"{ticker} Portfolio Performance", xaxis_title="Date", yaxis_title="Portfolio Value")
    fig.show()

    return portfolio

In [7]:
# Main script
if __name__ == "__main__":
    ticker = "INFY"
    start_date = "2020-01-01"
    end_date = "2024-01-01"
    short_window = 1
    long_window = 50

    data = get_stock_data(ticker, start_date, end_date)
    signals = moving_average_crossover(data, short_window, long_window)
    portfolio = backtest_strategy(data, signals)

    print(portfolio[['positions', 'market_price', 'cash', 'holding', 'total']].head())


[*********************100%***********************]  1 of 1 completed
C:\Users\tanya\AppData\Local\Temp\ipykernel_17124\256528663.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99030.06705267359' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  portfolio.iloc[i, portfolio.columns.get_loc('cash')] = cash
C:\Users\tanya\AppData\Local\Temp\ipykernel_17124\256528663.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99030.06705267359' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  portfolio.iloc[i, portfolio.columns.get_loc('total')] = cash + (position * price)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed